# Vision

The purpose of this notebook is to create images that will make the workflow of moving apples easy to do. 
The final product will be a Streamlit site where I can go forward and backwards and see frames and the transitions between them.

Requirements: Original frame, downsampled frame  with coordinates of every apple, previous downsampled frame, and a diff frame with the added apples in green and removed apples in red.

In [1]:
import cv2
import numpy as np
from pathlib import Path
import os

In [4]:
badapple_path = Path(".").resolve().parent / "badapple-small.mp4"

cap = cv2.VideoCapture(str(badapple_path))

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"width: {width}, height: {height}, fps: {fps}, n_frames: {n_frames}")

square_side = 30
black_threshold = 50

width: 960, height: 720, fps: 30, n_frames: 6572


In [3]:
# Generate real_frames
ret, frame = cap.read()
index = 0
while ret:
    cv2.imwrite(f"real_frames/{index}.jpg", frame)
    ret, frame = cap.read()
    index += 1

### Generate downsampled frames with coordinates of apples labeled

In [14]:
# reset cap
cap = cv2.VideoCapture(str(badapple_path))
index = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # convert to grayscale
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if index % 100 == 0 or index == n_frames - 1:
        print(f"frame {index}/{n_frames}", end="\r")
    out_frame = np.ones((height, width), dtype=np.uint8) * 255

    current_frame_pixels = set()
    for i in range(0, width - 1, square_side):
        for j in range(0, height - 1, square_side):
            section = frame[j: j + square_side, i: i + square_side]
            # if its dark, add dark circle with white coordinate text
            if (np.mean(section)) < black_threshold:
                cv2.circle(out_frame, (i+square_side//2, j+square_side//2), square_side//2, 0, -1)
                text_color = 255
            else:
                text_color = 0
            # otherwise, black coordinate text
            # put first coordinate higher, second coordinate lower
            
            cv2.putText(out_frame, f"{j//square_side+1}", (i+square_side//3-1, j + 22), cv2.FONT_HERSHEY_SIMPLEX, 0.3, text_color, 1, cv2.LINE_AA)
            cv2.putText(out_frame, f"{i//square_side+1}", (i+square_side//3-1, j + 12), cv2.FONT_HERSHEY_SIMPLEX, 0.3, text_color, 1, cv2.LINE_AA)
    cv2.imwrite(f"circle_frames/{index}.jpg", out_frame)
    index += 1

### Generate diff frames with additions in green and removals in red

In [15]:
# reset cap
cap = cv2.VideoCapture(str(badapple_path))
index = 0
last_frame_pixels = set()

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break
    # convert to grayscale
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if index % 100 == 0 or index == n_frames - 1:
        print(f"frame {index}/{n_frames}", end="\r")
        
    out_frame = np.ones((height, width, 3), dtype=np.uint8) * 255

    current_frame_pixels = set()
    for i in range(0, width - 1, square_side):
        for j in range(0, height - 1, square_side):
            section = frame[j: j + square_side, i: i + square_side]
             # three cases we care about: a pixel is persisted, a pixel is added, a pixel is removed
            circle_colour = None
            if (np.mean(section)) < 50:
                current_frame_pixels.add((i, j))
                # black pixel
                if (i, j) in last_frame_pixels:
                    # persisted from previous frame. Black circle, white text
                    circle_colour = (0, 0, 0)
                    text_color = (255, 255, 255)
                else:
                    # new black dot. Green circle, black text
                    circle_colour = (0, 255, 0)
                    text_color = (0, 0 ,0)
            else:
                # white pixel
                if (i, j) in last_frame_pixels:
                    # removed black dot. Red circle, black text
                    circle_colour = (0, 0, 255)
                    text_color = (0, 0, 0)
                else:
                    # persisted nothing. Just use black text
                    text_color = (0, 0 ,0)
                    
            if circle_colour is not None:
                cv2.circle(out_frame, (i+square_side//2, j+square_side//2), square_side//2, circle_colour, -1)
            cv2.putText(out_frame, f"{j//square_side+1}", (i+square_side//3-1, j + 22), cv2.FONT_HERSHEY_SIMPLEX, 0.3, text_color, 1, cv2.LINE_AA)
            cv2.putText(out_frame, f"{i//square_side+1}", (i+square_side//3-1, j + 12), cv2.FONT_HERSHEY_SIMPLEX, 0.3, text_color, 1, cv2.LINE_AA)
    cv2.imwrite(f"diff_frames/{index}.jpg", out_frame)

    last_frame_pixels = current_frame_pixels
    index += 1